### 성공! (평행선 그룹 간 intersections 자동으로 구해서 4방향 구분) ==> 함수로 쪼개기
참고 코드 : https://java2020.com/q/ktdcksju

In [1]:
import cv2
import numpy as np
import math

# 1. Otsu
src = cv2.imread('test_resize.png', cv2.IMREAD_GRAYSCALE)
cv2.imshow('src', src)

ret, otsu = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imshow('otsu', otsu)

# 2. Morphology
mask = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
erode = cv2.erode(otsu, mask, iterations = 1)
dilate = cv2.dilate(erode, mask, iterations = 2) # 한번 더 팽창해서 중간 구멍 제거
cv2.imshow('erode', erode)
cv2.imshow('dilate', dilate)

# 3
edge = cv2.Canny(dilate, 50, 150)
lines = cv2.HoughLines(edge, 1, math.pi / 180, 114) # threshold를 250 -> 114로!!

print(lines) # 직선정보(rho, theta)를 저장할 출력 벡터! (네점의 좌표 X)

points = []

if lines is not None:
    for i in range(lines.shape[0]):
        c1 = lines[(i+4)%4][0][0] # rho1 
        c2 = lines[(i+5)%4][0][0] # rho2
        theta1 = lines[(i+4)%4][0][1]
        theta2 = lines[(i+5)%4][0][1]
        a1 = math.cos(theta1)
        a2 = math.cos(theta2)
        b1 = math.sin(theta1)
        b2 = math.sin(theta2)
        
        x = int((b2*c1 - b1*c2) / (a1*b2 - a2*b1))
        y = int((c1 - a1*x) / b1)
        
        points.append([x, y])

print(points)

cv2.waitKey()
cv2.destroyAllWindows()

[[[-162.           2.5481806]]

 [[ -31.           2.9321532]]

 [[ 352.           1.2566371]]

 [[  82.           1.3613569]]]
[[-43, -353], [103, 335], [2547, -457], [220, 37]]


In [2]:
from collections import defaultdict
def segment_by_angle_kmeans(lines, k=2, **kwargs):
    """Groups lines based on angle with k-means.

    Uses k-means on the coordinates of the angle on the unit circle 
    to segment `k` angles inside `lines`.
    """

    # Define criteria = (type, max_iter, epsilon)
    default_criteria_type = cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER
    criteria = kwargs.get('criteria', (default_criteria_type, 10, 1.0))
    flags = kwargs.get('flags', cv2.KMEANS_RANDOM_CENTERS)
    attempts = kwargs.get('attempts', 10)

    # returns angles in [0, pi] in radians
    angles = np.array([line[0][1] for line in lines])
    # multiply the angles by two and find coordinates of that angle
    pts = np.array([[np.cos(2*angle), np.sin(2*angle)]
                    for angle in angles], dtype=np.float32)

    # run kmeans on the coords
    labels, centers = cv2.kmeans(pts, k, None, criteria, attempts, flags)[1:]
    labels = labels.reshape(-1)  # transpose to row vec

    # segment lines based on their kmeans label
    segmented = defaultdict(list)
    for i, line in zip(range(len(lines)), lines):
        segmented[labels[i]].append(line)
    segmented = list(segmented.values())
    return segmented

In [3]:
segmented = segment_by_angle_kmeans(lines)
print(segmented)

[[array([[-162.       ,    2.5481806]], dtype=float32), array([[-31.       ,   2.9321532]], dtype=float32)], [array([[352.       ,   1.2566371]], dtype=float32), array([[82.       ,  1.3613569]], dtype=float32)]]


In [4]:
def intersection(line1, line2):
    """Finds the intersection of two lines given in Hesse normal form.

    Returns closest integer pixel locations.
    See https://stackoverflow.com/a/383527/5087436
    """
    rho1, theta1 = line1[0]
    rho2, theta2 = line2[0]
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]


def segmented_intersections(lines):
    """Finds the intersections between groups of lines."""

    intersections = []
    for i, group in enumerate(lines[:-1]):
        for next_group in lines[i+1:]:
            for line1 in group:
                for line2 in next_group:
                    intersections.append(intersection(line1, line2)) 

    return intersections

In [5]:
intersections = segmented_intersections(segmented)
print(intersections) # intersections[0][0] = [365, 252]

[[[365, 252]], [[220, 37]], [[103, 337]], [[47, 74]]]


In [6]:
# y값 기준 오름차순 정렬
intersections.sort(key = lambda x : x[0][1])
print(intersections)

# x값 기준 오름차순 정렬
intersections.sort(key = lambda x : x[0][0])
print(intersections)

[[[220, 37]], [[47, 74]], [[365, 252]], [[103, 337]]]
[[[47, 74]], [[103, 337]], [[220, 37]], [[365, 252]]]


In [7]:
# 시계 방향으로 좌표 정렬 (반으로 나눠서 크기 비교 두번씩)
intersections_sorted = np.zeros([4, 2], dtype = np.float32)
intersections.sort(key = lambda x : x[0][1]) # y값 기준 오름차순 정렬

if intersections[0][0][0] < intersections[1][0][0]:
    lu = intersections[0][0]
    ru = intersections[1][0]
else:
    lu = intersections[1][0]
    ru = intersections[0][0]
    
if intersections[2][0][0] < intersections[3][0][0]:
    rd = intersections[3][0]
    ld = intersections[2][0]
else:
    rd = intersections[2][0]
    ld = intersections[3][0]
    
    
intersections_sorted[0, :] = np.array(lu).astype(np.float32)
intersections_sorted[1, :] = np.array(ru).astype(np.float32)
intersections_sorted[2, :] = np.array(rd).astype(np.float32)
intersections_sorted[3, :] = np.array(ld).astype(np.float32)

print(intersections_sorted)

[[ 47.  74.]
 [220.  37.]
 [365. 252.]
 [103. 337.]]


In [9]:
# 점으로 각 꼭지점 표시
for i in range(4):
    cv2.circle(src, (intersections_sorted[i][0], intersections_sorted[i][1]), 5, (0, 0, 255), -1) # 클릭 시 빨간색 원
    
cv2.imshow('circle', src)
cv2.waitKey()
cv2.destroyAllWindows()

![](result9.png)

In [10]:
# 5. 투시 변환
w = 400 # dst 크기 사전에 지정
h = 600
            
# 좌상단부터 시계 방향으로 처리
dst_pts = np.array([[0, 0],
                    [w - 1, 0],
                    [w - 1, h - 1],
                    [0, h - 1]]).astype(np.float32)
            
pers_mat = cv2.getPerspectiveTransform(intersections_sorted, dst_pts) # M 구하기

src = cv2.imread('test_resize.png')

result = cv2.warpPerspective(src, pers_mat, (w, h)) # M 이용해 투시 변환
            
cv2.imshow('result', result)


cv2.waitKey()
cv2.destroyAllWindows()

![](result10.png)

<hr>